In [1]:
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# --- Text Preprocessing Imports ---
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

# Download stopwords once
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# --- 1. Setup Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# --- 2. Define Preprocessing Function ---
def preprocess_text(text):
    text = str(text) 
    # 1. Removal of HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 2. To Lower Case
    text = text.lower()
    
    # 3. Removal of URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 4. Removal of Twitter Handles (@user)
    text = re.sub(r'@\w+', '', text)
    
    # 5. Removal of Hashtag symbol (keeping text)
    text = re.sub(r'#', '', text) 
    
    # 6. Removal of Placeholders
    text = re.sub(r'\[.*?\]', '', text)

    # 7. Removal of Punctuation & Non-letter Characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 8. Removal of Stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # 9. Clean extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("✅ Setup & Preprocessing Function Ready!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nabil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🚀 Device: cuda
✅ Setup & Preprocessing Function Ready!


In [2]:
class MultimodalDisasterClassifier(nn.Module):
    def __init__(self, model_id, num_classes=2):
        super(MultimodalDisasterClassifier, self).__init__()
        
        print(f"Loading {model_id} with SafeTensors...")
        self.clip = CLIPModel.from_pretrained(model_id, use_safetensors=True)
        
        # Increased capacity in the head (Standard 512 -> 256 -> 2)
        self.classifier = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),   # Added BatchNorm for stability
            nn.ReLU(),
            nn.Dropout(0.3),       # Increased Dropout to 0.3
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        text_out = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        img_out = self.clip.get_image_features(pixel_values=pixel_values)
        
        # Normalize
        text_out = text_out / text_out.norm(dim=-1, keepdim=True)
        img_out = img_out / img_out.norm(dim=-1, keepdim=True)
        
        combined = torch.cat((img_out, text_out), dim=1)
        logits = self.classifier(combined.float())
        return logits


In [3]:
# --- 3. Define Custom Dataset Class ---
class CrisisDataset(Dataset):
    def __init__(self, df, processor, data_path="."):
        self.df = df
        self.processor = processor
        self.data_path = data_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['tweet_text']
        label = row['label']
        img_path = os.path.join(self.data_path, row['image'])
        
        try:
            # Load and convert image
            image = Image.open(img_path).convert("RGB")
            
            # CLIP Processor handles Image Resizing/Norm + Text Tokenization
            encoding = self.processor(
                text=text, 
                images=image, 
                return_tensors="pt", 
                padding="max_length", 
                truncation=True, 
                max_length=77
            )
            
            # Remove batch dimension added by processor
            return {
                'pixel_values': encoding['pixel_values'].squeeze(0),
                'input_ids': encoding['input_ids'].squeeze(0),
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'label': torch.tensor(label, dtype=torch.long)
            }
        except Exception as e:
            # If an image fails, return the next one (simple error handling)
            return self.__getitem__((idx + 1) % len(self.df))

print("✅ Dataset Class Defined!")

✅ Dataset Class Defined!


In [ ]:
# --- 4. Load & Clean Data ---
# Define paths (Adjust these to match your folder structure)
train_path = os.path.join("data/CrisisMMD/crisismmd_datasplit_all/crisismmd_datasplit_agreed_label/task_informative_text_img_agreed_lab_train.tsv")
test_path = os.path.join("data/CrisisMMD/crisismmd_datasplit_all/crisismmd_datasplit_agreed_label/task_informative_text_img_agreed_lab_test.tsv")

# Load DataFrames
train_df = pd.read_csv(train_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')

# Function to encode labels
def encode_label(row):
    if row['label_image'] == 'informative': return 1
    elif row['label_image'] == 'not_informative': return 0
    return None

# Apply Label Encoding
train_df['label'] = train_df.apply(encode_label, axis=1)
test_df['label'] = test_df.apply(encode_label, axis=1)

# Drop NaNs
train_df = train_df.dropna(subset=['label'])
test_df = test_df.dropna(subset=['label'])
train_df['label'] = train_df['label'].astype(int)
test_df['label'] = test_df['label'].astype(int)

# --- APPLY PREPROCESSING HERE ---
print("⏳ Applying Text Preprocessing (Steps 1-9)...")
tqdm.pandas() # Progress bar
train_df['tweet_text'] = train_df['tweet_text'].progress_apply(preprocess_text)
test_df['tweet_text'] = test_df['tweet_text'].progress_apply(preprocess_text)

# Initialize Processor
model_id = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(model_id)

# Create Datasets & Loaders
batch_size = 32 # Keep small for GPU memory
train_dataset = CrisisDataset(train_df, processor)
test_dataset = CrisisDataset(test_df, processor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"✅ Data Ready! Train: {len(train_df)}, Test: {len(test_df)}")

⏳ Applying Text Preprocessing (Steps 1-9)...


100%|██████████| 2237/2237 [00:00<00:00, 12567.38it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Data Ready! Train: 13608, Test: 2237


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from peft import LoraConfig, get_peft_model
from tqdm import tqdm

# --- A. RE-INITIALIZE YOUR BASE MODEL (CRITICAL) ---
# RESTART YOUR KERNEL BEFORE RUNNING THIS.
# Replace "your-model-id-here" with your actual model path (e.g., "openai/clip-vit-base-patch32")
MODEL_ID = "openai/clip-vit-base-patch32" # <-- UPDATE THIS ACCORDING TO YOUR THESIS CODE

model = MultimodalDisasterClassifier(model_id=MODEL_ID, num_classes=2) 
model.to("cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

# --- B. DEFINE HELPERS ---
class MockConfig:
    def __init__(self):
        self.tie_word_embeddings = False
        self.use_return_dict = False
    def get(self, key, default=None):
        return getattr(self, key, default)

class EarlyStopper:
    def __init__(self, patience=4):
        self.patience, self.counter, self.best_acc, self.early_stop = patience, 0, 0.0, False
    def __call__(self, val_acc):
        if val_acc > self.best_acc:
            self.best_acc, self.counter = val_acc, 0
            return True
        self.counter += 1
        if self.counter >= self.patience: self.early_stop = True
        return False

# --- C. APPLY CLEAN LoRA ---
model.config = MockConfig()
lora_config = LoraConfig(
    r=64, 
    lora_alpha=128, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], 
    lora_dropout=0.1, 
    bias="none"
)

# 
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# --- D. TRAINING SETUP ---
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
early_stopper = EarlyStopper(patience=4)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)


Loading openai/clip-vit-base-patch32 with SafeTensors...
trainable params: 5,898,240 || all params: 157,767,299 || trainable%: 3.7386


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from peft import LoraConfig, get_peft_model
from transformers import CLIPModel, get_cosine_schedule_with_warmup
from torch.amp import GradScaler, autocast 
from tqdm import tqdm

# --- 1. HELPER CLASSES ---

class MockConfig:
    """Fixes PEFT compatibility issues."""
    def __init__(self):
        self.tie_word_embeddings = False
        self.use_return_dict = False
    def get(self, key, default=None):
        return getattr(self, key, default)

class EarlyStopper:
    """Monitors validation accuracy to prevent overfitting."""
    def __init__(self, patience=5):
        self.patience, self.counter, self.best_acc, self.early_stop = patience, 0, 0.0, False
    def __call__(self, val_acc):
        if val_acc > self.best_acc:
            self.best_acc, self.counter = val_acc, 0
            return True
        self.counter += 1
        if self.counter >= self.patience: self.early_stop = True
        return False

# --- 2. GATED ARCHITECTURE (The Technical Novelty) ---

class GatedMultimodalClassifier(nn.Module):
    """Uses Adaptive Gating to weight modalities dynamically."""
    def __init__(self, model_id="openai/clip-vit-base-patch32", num_classes=2):
        super().__init__()
        # use_safetensors=True bypasses 2025 security vulnerability
        self.clip = CLIPModel.from_pretrained(model_id, use_safetensors=True)
        
        # Adaptive Gate: Scales features based on their reliability
        self.gate = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.Sigmoid()
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        text_out = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        img_out = self.clip.get_image_features(pixel_values=pixel_values)
        
        # L2 Normalization ensures features are on the same scale
        text_f = text_out / text_out.norm(dim=-1, keepdim=True)
        img_f = img_out / img_out.norm(dim=-1, keepdim=True)
        
        combined = torch.cat((img_f, text_f), dim=1)
        # Gating mechanism suppresses noise in disparate modalities
        gated_f = combined * self.gate(combined.float())
        
        return self.classifier(gated_f)

# --- 3. INITIALIZATION & DoRA ---

NUM_CLASSES = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GatedMultimodalClassifier(num_classes=NUM_CLASSES).to(device)
model.config = MockConfig()

lora_config = LoraConfig(
    r=32, lora_alpha=64, use_dora=True, # DoRA matches Full FT performance
    target_modules=["q_proj", "v_proj", "fc1", "fc2"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["LayerNorm", "ln_1", "ln_2", "ln_final", "classifier", "gate"]
)

model = get_peft_model(model, lora_config)
print(f"✅ Gated Model Ready. Trainable Params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# --- 4. OPTIMIZER & SCHEDULER ---

EPOCHS = 15
total_steps = len(train_loader) * EPOCHS

# Differential Learning Rates stabilize the fine-tuning process
optimizer = optim.AdamW([
    {'params': [p for n, p in model.named_parameters() if "classifier" not in n and "gate" not in n], 'lr': 2e-5},
    {'params': model.classifier.parameters(), 'lr': 8e-5},
    {'params': model.gate.parameters(), 'lr': 1e-4} 
], weight_decay=0.05)

scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=int(0.15 * total_steps), num_training_steps=total_steps
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1) 
scaler = GradScaler('cuda')
early_stopper = EarlyStopper(patience=5)

# --- 5. TRAINING LOOP ---



best_acc = 0.0
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for batch in loop:
        ids, mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        pixels, labels = batch['pixel_values'].to(device), batch['label'].to(device)
        
        optimizer.zero_grad()
        with autocast('cuda'): 
            outputs = model(ids, mask, pixels)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        total_loss += loss.item()
        loop.set_postfix(loss=f"{loss.item():.4f}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad(), autocast('cuda'):
        for batch in test_loader:
            v_ids, v_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            v_pixels, v_labels = batch['pixel_values'].to(device), batch['label'].to(device)
            out = model(v_ids, v_mask, v_pixels)
            correct += (torch.argmax(out, 1) == v_labels).sum().item()
            total += v_labels.size(0)
    
    val_acc = correct / total
    print(f"📉 Epoch {epoch+1} Val Acc: {val_acc:.4f} | Avg Loss: {total_loss/len(train_loader):.4f}")

    if early_stopper(val_acc):
        best_acc = val_acc
        torch.save(model.state_dict(), "best_informativeness_gated.pth")
        print("⭐ PERFORMANCE BREAKTHROUGH: New Best Model Saved!")

    if early_stopper.early_stop:
        print(f"🛑 Stopping Early. Best Accuracy: {best_acc:.4f}")
        break

print(f"\n🏆 Final Result: {best_acc:.4f}")

✅ Gated Model Ready. Trainable Params: 8565250


Epoch 1/15: 100%|██████████| 426/426 [09:55<00:00,  1.40s/it, loss=0.2273]


📉 Epoch 1 Val Acc: 0.8869 | Avg Loss: 0.5183
⭐ PERFORMANCE BREAKTHROUGH: New Best Model Saved!


Epoch 2/15: 100%|██████████| 426/426 [12:48<00:00,  1.80s/it, loss=0.4650]


📉 Epoch 2 Val Acc: 0.8891 | Avg Loss: 0.3853
⭐ PERFORMANCE BREAKTHROUGH: New Best Model Saved!


...

📉 Epoch 5 Val Acc: 0.8869 | Avg Loss: 0.2507


Epoch 6/15: 100%|██████████| 426/426 [13:10<00:00,  1.86s/it, loss=0.2348]


📉 Epoch 6 Val Acc: 0.8833 | Avg Loss: 0.2314


Epoch 7/15: 100%|██████████| 426/426 [13:35<00:00,  1.92s/it, loss=0.2261]


📉 Epoch 7 Val Acc: 0.8815 | Avg Loss: 0.2228
🛑 Stopping Early. Best Accuracy: 0.8891

🏆 Final Result: 0.8891


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from peft import LoraConfig, get_peft_model
from transformers import CLIPModel, get_cosine_schedule_with_warmup
from torch.amp import GradScaler, autocast 
import numpy as np
from tqdm import tqdm

# --- 1. HELPER CLASSES ---

class MockConfig:
    def __init__(self):
        self.tie_word_embeddings = False
        self.use_return_dict = False
    def get(self, key, default=None):
        return getattr(self, key, default)

class EarlyStopper:
    def __init__(self, patience=5):
        self.patience, self.counter, self.best_acc, self.early_stop = patience, 0, 0.0, False
    def __call__(self, val_acc):
        if val_acc > self.best_acc:
            self.best_acc, self.counter = val_acc, 0
            return True
        self.counter += 1
        if self.counter >= self.patience: self.early_stop = True
        return False

class StableFocalLoss(nn.Module):
    """Focuses on the final 10% of 'hard' ambiguous disaster tweets."""
    def __init__(self, gamma=3.0, alpha=0.25):
        super().__init__()
        self.gamma, self.alpha = gamma, alpha
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        return (self.alpha * (1 - pt)**self.gamma * ce_loss).mean()

# --- 2. ELITE ARCHITECTURE: SYMMETRIC DUAL-PATH ATTENTION ---

class SymmetricBCAFusion(nn.Module):
    """Implements mutual grounding between text and visual features."""
    def __init__(self, embed_dim=512, heads=8):
        super().__init__()
        self.text_to_img = nn.MultiheadAttention(embed_dim, heads, batch_first=True)
        self.img_to_text = nn.MultiheadAttention(embed_dim, heads, batch_first=True)
        self.gate = nn.Sequential(nn.Linear(embed_dim * 2, embed_dim * 2), nn.Sigmoid())
        self.norm = nn.LayerNorm(embed_dim * 2)

    def forward(self, text_f, img_f):
        # Path A: Text finds evidence in Image
        aligned_t, _ = self.text_to_img(text_f.unsqueeze(1), img_f.unsqueeze(1), img_f.unsqueeze(1))
        # Path B: Image finds context in Text
        aligned_i, _ = self.img_to_text(img_f.unsqueeze(1), text_f.unsqueeze(1), text_f.unsqueeze(1))
        
        combined = torch.cat((aligned_t.squeeze(1), aligned_i.squeeze(1)), dim=1)
        # Residual Gating: Stabilizes learning and prevents feature suppression
        return self.norm(combined * self.gate(combined) + torch.cat((text_f, img_f), dim=1))

class EliteDualFusionClassifier(nn.Module):
    def __init__(self, model_id="openai/clip-vit-base-patch32", num_classes=2):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(model_id, use_safetensors=True)
        self.fusion = SymmetricBCAFusion()
        self.classifier = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.GELU(),
            nn.Dropout(0.5), nn.Linear(512, 128), nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        t_f = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        i_f = self.clip.get_image_features(pixel_values=pixel_values)
        t_f, i_f = t_f / t_f.norm(dim=-1, keepdim=True), i_f / i_f.norm(dim=-1, keepdim=True)
        return self.classifier(self.fusion(t_f, i_f))

# --- 3. TRAINING INITIALIZATION ---

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EliteDualFusionClassifier().to(device)
model.config = MockConfig()

lora_config = LoraConfig(
    r=64, lora_alpha=128, use_dora=True, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "fc1", "fc2"],
    lora_dropout=0.1, modules_to_save=["LayerNorm", "classifier", "fusion"]
)
model = get_peft_model(model, lora_config)

optimizer = optim.AdamW([
    {'params': [p for n, p in model.named_parameters() if not any(x in n for x in ["classifier", "fusion"])], 'lr': 5e-6},
    {'params': model.classifier.parameters(), 'lr': 1e-4}, 
    {'params': model.fusion.parameters(), 'lr': 1e-4}
], weight_decay=0.15) # High decay forces the model to generalize

scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=int(0.1 * len(train_loader)*15), num_training_steps=len(train_loader)*15
)
criterion = StableFocalLoss(gamma=3.0) 
scaler = GradScaler('cuda')
early_stopper = EarlyStopper(patience=4)

# --- 4. THE PUSH TO 91% ---


print("🚀 Starting the Symmetric Fusion Elite Run...")
best_acc = 0.0
for epoch in range(15):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        ids, mask, pixels, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['pixel_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        with autocast('cuda'):
            loss = criterion(model(ids, mask, pixels), labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        total_loss += loss.item()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad(), autocast('cuda'):
        for batch in test_loader:
            out = model(batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['pixel_values'].to(device))
            correct += (torch.argmax(out, 1) == batch['label'].to(device)).sum().item()
            total += batch['label'].size(0)
    
    val_acc = correct / total
    print(f"📉 Epoch {epoch+1} Val Acc: {val_acc:.4f} | Loss: {total_loss/len(train_loader):.4f}")

    if early_stopper(val_acc):
        best_acc = val_acc
        torch.save(model.state_dict(), "best_symmetric_fusion.pth")
        print("⭐ BREAKTHROUGH: Best Model Saved.")

    if early_stopper.early_stop: break

print(f"🏆 Final Result: {best_acc:.4f}")

🚀 Starting the Symmetric Fusion Elite Run...


Epoch 1:  95%|█████████▌| 406/426 [21:00<01:06,  3.33s/it]d:\Anaconda\envs\tweet_project\lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1: 100%|██████████| 426/426 [22:05<00:00,  3.11s/it]


📉 Epoch 1 Val Acc: 0.8891 | Loss: 0.0146
⭐ BREAKTHROUGH: Best Model Saved.


Epoch 2: 100%|██████████| 426/426 [20:13<00:00,  2.85s/it]


📉 Epoch 2 Val Acc: 0.8918 | Loss: 0.0096
⭐ BREAKTHROUGH: Best Model Saved.


...

📉 Epoch 4 Val Acc: 0.8829 | Loss: 0.0061


Epoch 5: 100%|██████████| 426/426 [18:52<00:00,  2.66s/it]


📉 Epoch 5 Val Acc: 0.8824 | Loss: 0.0044


Epoch 6: 100%|██████████| 426/426 [16:36<00:00,  2.34s/it]


📉 Epoch 6 Val Acc: 0.8820 | Loss: 0.0030
🏆 Final Result: 0.8918


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import torch

# --- 1. EVALUATION DATA COLLECTION ---
def get_predictions(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad(), autocast('cuda'):
        for batch in loader:
            ids, mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            pixels, labels = batch['pixel_values'].to(device), batch['label'].to(device)
            
            outputs = model(ids, mask, pixels)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy()[:, 1]) # Probability for 'Informative' class
            
    return np.array(all_preds), np.array(all_labels), np.array(all_probs)

# Load the best weights before evaluation
model.load_state_dict(torch.load("best_symmetric_fusion.pth"))
y_pred, y_true, y_probs = get_predictions(model, test_loader, device)

# --- 2. PLOT CONFUSION MATRIX ---
# Essential for identifying which disaster tweets the model still finds 'ambiguous'
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Informative', 'Informative'],
            yticklabels=['Non-Informative', 'Informative'])
plt.title('Confusion Matrix: Symmetric Dual-Path Attention')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

# --- 3. PLOT ROC CURVE ---
# Proves the model's 'Situational Awareness' across varying thresholds
fpr, tpr, _ = roc_curve(y_true, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# --- 4. CLASSIFICATION REPORT ---
print("\n📝 Detailed Performance Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Informative', 'Informative']))

C:\Users\nabil\AppData\Local\Temp\ipykernel_14276\3891510328.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_symmetric_fusion.pth